In [ ]:
#Run this code first. You will need to copy and paste credentials into colab
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials
import os
import pandas as pd
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

download_path = os.path.expanduser('~/data')
try:
  os.makedirs(download_path)
except FileExistsError:
  pass

In [1]:
def find_files(contains_string):
    #Enter string to find files in google drive that match that string
    #Returns list of file names/titles and ids that match that string
    #Enter exact file name to get just that file
    #Note that this will also search your trash in drive
    listed = drive.ListFile({'q': f"title contains '{contains_string}'"}).GetList()
    ids = []
    titles = []
    for file in listed:
        print('title {}, id {}'.format(file['title'], file['id']))
        ids.append(file['id'])
        titles.append(file['title'])
        
    return title, ids

In [ ]:
def get_file(title, ID):
    #Enter title and id found from find_files function
    #Returns file_string that can be used for loading files
    #E.g. pd.read_csv(file_string) or np.load(file_string)
    global download_path
    output_file = os.path.join(download_path, title)
    temp_file = drive.CreateFile({'id' : ID})
    temp_file.GetContentFile(output_file)
    
    return output_file

In [ ]:
def save_file_to_drive(filename, verbose=True):
    #Saves file to drive
    #File should already be saved in colab before passing filename to this function
    #E.g. run pd.to_csv(filename) or np.save(filename, array) before this function
    
    uploaded = drive.CreateFile({'title': filename})
    uploaded.SetContentFile(filename)
    uploaded.Upload()
    
    if verbose:
        print(f"Uploaded file with ID {uploaded.get('id')}")